In [203]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
import time
import datetime
import re
import pickle

In [225]:
### preprocessing 

def split_category(value):
    vals = []
    if '|' in value:
        vals = value.split('|')
    else:
        vals.append(value)
    return vals

def save_unique(data, col, filename):
    unique_vals = data[col].unique()
    unique_vals = [i.title() for i in unique_vals]
    unique_vals = pd.DataFrame(unique_vals, columns=[col])
    unique_vals.to_csv(filename, index=False)

def get_all_values_from_airline(data):
    airlines = []
    airlines = list(set(data['Airline']))
    airlines = [i.title() for i in airlines]
    print(airlines) 

def split_columns_with_multiple_values(data):
    data['Airline'] = data['Airline'].apply(split_category)
    data['Places Covered'] = data['Places Covered'].apply(split_category)
    data['Sightseeing Places Covered'] = data['Sightseeing Places Covered'].apply(split_category)
    data['Hotel Details'] = data['Hotel Details'].apply(split_category)    
    
    prepare_itinerary(data)

    save_unique(data, 'Airline', 'airlines.csv')

    
    return data

def one_hot_encoding_on_columns(data):
    for col in data.columns:
        if col in [
            'Itinerary',
            'Sightseeing Places Covered',
            'Places Covered',
            'Hotel Details',
            'Airline',
        ]:
            encoder = MultiLabelBinarizer()
            # df = pd.DataFrame(encoder.fit(data[col]), columns=encoder.classes_)
            encoder.fit(data[col])
            np.save('encoder_{0}.npy'.format(col.replace(" ", "_")), encoder.classes_, allow_pickle=True)
            df = pd.DataFrame(encoder.transform(data[col]), columns=encoder.classes_)
            data.drop(columns=[col], inplace=True)
            data = pd.concat([data, df], axis=1)    
            
        elif data[col].dtype == 'object':
            encoder = OneHotEncoder()
            encoder.fit(data[col].values.reshape(-1, 1))
            np.save('encoder_{0}.npy'.format(col.replace(" ", "_")), encoder.categories_, allow_pickle=True)
            dummy = pd.DataFrame(encoder.transform(data[col].values.reshape(-1, 1)).toarray())
            data.drop(columns=[col], inplace=True)
            data = pd.concat([data, dummy], axis=1)

    return data

def show_category(series):
    values = {}
    for val in series:
        for each in val:
            if each in values:
                values[each] += 1
            else:
                values[each] = 1
    return values

def make_feature_col(series, all_keys):
    feature_dict = {key: [] for key in all_keys}
    for items in series:
        for key in all_keys:
            if key not in items:
                feature_dict[key].append(0)
            else:
                feature_dict[key].append(1)

    return pd.DataFrame(feature_dict)

def structuring_columns(data):
    change = lambda pc: [each+'_ae' for each in pc]
    A_all_keys = show_category(data['Airline']).keys()
    A_all_keys = change(A_all_keys)
    airline = make_feature_col(data['Airline'], A_all_keys)
    data = pd.concat([data, airline], axis=1)
    
    change = lambda pc: [each+'_pc' for each in pc]
    PC_all_keys = show_category(data['Places Covered']).keys()
    PC_all_keys = change(PC_all_keys)
    p_covered = make_feature_col(data['Places Covered'], PC_all_keys)
    data = pd.concat([data, p_covered], axis=1)

    change = lambda pc: [each+'_hd' for each in pc]
    HD_all_keys = show_category(data['Hotel Details']).keys()
    HD_all_keys = change(PC_all_keys)
    h_details = make_feature_col(data['Hotel Details'], HD_all_keys)
    data = pd.concat([data, h_details], axis=1)

    return data

def prepare_itinerary(data):
    data['Itinerary'] = data['Itinerary'].str.split(".")
    vals = data['Itinerary']
    for j in range(len(vals)):
        for p in range(len(vals[j])):
            vals[j][p] = re.sub(r'[^0-9N]', '', vals[j][p])
            vals[j][p] = vals[j][p].split('N')
            vals[j][p] = list(filter(None, vals[j][p]))
            vals[j][p] = [int(i) for i in vals[j][p]]
        vals[j] = [item for sublist in vals[j] for item in sublist]
    data['Itinerary'] = vals 

def remove_useless_columns(data):
    data.drop('Uniq Id', inplace=True, axis=1)
    data.drop('Package Name', inplace=True, axis=1)
    data.drop('Destination', inplace=True, axis=1)
    data.drop('Cancellation Rules', inplace=True, axis=1)
    return data

def prepare_hotel_details(data):
    data['Hotel Details'] = [re.sub(r':.*', '', i) for i in data['Hotel Details']]
    data['Hotel Details'] = [i.lower() for i in data['Hotel Details']]
    data['Hotel Details'] = data['Hotel Details'].str.replace(" - mmt holidays special",'')
    data['Hotel Details'] = data['Hotel Details'].str.replace("- mmt holidays special",'')
    data['Hotel Details'] = data['Hotel Details'].str.replace("-mmt holidays special",'')
    data['Hotel Details'] = data['Hotel Details'].str.replace("(mmt holidays special)",'')
    data['Hotel Details'] = data['Hotel Details'].str.replace(" - mmt special",'')
    data['Hotel Details'] = data['Hotel Details'].str.replace("- mmt speciall",'')
    data['Hotel Details'] = data['Hotel Details'].str.replace("-mmt special",'')
    data['Hotel Details'] = data['Hotel Details'].str.replace("mmt special",'')
  
    hotel_details = data['Hotel Details'].unique()
    hotel_details = [i.title() for i in hotel_details]
    hotel_details = pd.DataFrame(hotel_details, columns=['Hotel Details'])
    hotel_details.to_csv('hotel_details.csv', index=False)

    return data

def convert_dates_to_one_format(data):
    data['Travel Date'] = pd.to_datetime(data['Travel Date'], errors='coerce')
    data['Travel Date'] = data['Travel Date'].astype(np.int64) // 10**9    
    return data

def preprocess_data(data):
    data = remove_useless_columns(data)
    data = convert_dates_to_one_format(data)
    data = prepare_hotel_details(data)
    data = split_columns_with_multiple_values(data)
    data = one_hot_encoding_on_columns(data)
    # data = structuring_columns(data)
    return(data)

In [226]:
train = pd.read_csv('dataset\Train.csv')
validation = pd.read_csv('dataset\Test.csv')

In [227]:
final_train_data = preprocess_data(train)
final_validation_data = preprocess_data(validation)

TypeError: unhashable type: 'list'

In [ ]:
### Displaying processed data

print('================TRAINING DATA==================')
display(final_train_data)

================TRAINING DATA==================


,Travel Date,Flight Stops,Meals,Per Person Price,0,1,2,3,4,1,2,3,4,5,6,7,8,Abu Dhabi,Agra,Ahmedabad,Ahmednagar,Ajmer,Alibagh,Allepey,Almora,Alwar,Amritsar,Anuradhapura,Auckland,Aurangabad,Bali,Bandipur,Bangalore,Bangkok,Batam,Bentota,Bharatpur,Bikaner,Bintan Island,Bundi,Cairns,Chail,Chandigarh,Chennai,Cherrapunjee,Chikmangalur,Chintpurni,Chittorgarh,Christchurch,Cochin,Colombo,Coorg,Copenhagen,Corbett,Dalhousie,Daman,Dambulla,Darjeeling,Dharamshala,Dubai,Dwarka,Franz Josef,Galle,Gangtok,Ganpatipule,Genting,Goa,Gokarna,Gold Coast,Gulmarg,Guwahati,Gwalior,Hampi,Haridwar,Hassan,Havelock,Helsinki,Hong Kong,Jaipur,Jaisalmer,Jakarta,Jammu,Jamnagar,Jodhpur,Johor Bahru,Joshimath,Kabini,Kalimpong,Kanchipuram,Kandy,Kangra,Kanyakumari,Kasauli,Kasol,Katra,Kausani,Kaziranga,Khajuraho,Kishangarh,Kodaikanal,Kota Kinabalu,Kovalam,Kovalam and Poovar,Krabi town,Kuala Lumpur,Kullu,Kumarakom,Kumbakonam,Kumbalgarh,Kuta,Lachen,Lachung,Langkawi,Leh,Lonavala And Khandala,Lucknow,Madurai,Mahabalipuram,Mahe Island,Manali,Mandawa,Mathura,Mauritius,Melbourne,Moscow,Mount Abu,Mumbai,Munnar,Mussoorie,Mysore,Nainital,Namchi,Nameri,Nanded,Nasik,Nawalgarh,Negombo,New Delhi,Niel Island,Nubra Valley,Nuwara Eliya,Ooty,Orchha,Oslo,Pahalgam,Palampur,Pali,Paris,Paro,Parwanoo,Patnitop,Pattaya,Pelling,Penang,Phuentsholing,Phuket,Pondicherry,Poovar,Port Blair,Prague,Praslin Island,Punakha,Pushkar,Queenstown,Rameshwaram,Ranakpur,Ranikhet,Ranthambhore,Ras Al Khaimah,Rishikesh,Rotorua,Saint Petersburg,Saputara,Seminyak,Sharjah,Shillong,Shimla,Shirdi,Sigiriya,Siliguri,Singapore,Singapore Cruise,Somnath,Sonmarg,Srinagar,Sydney,Tanjore,Thekkady,Thimpu,Tiruchirapally,Tirupati,Trincomalee,Trivandrum,Udaipur,Vadodara,Varanasi,Varkala,Wayanad,Yala,175 metropolitan executive motel on riccarton,9hotel montparnasse,a one star hotel pattaya,aahana - the corbett wilderness,aamod at dalhousie,abad whispering palms,abhishek beach resort and spa,achrol niwas a treehouse resort,acron waterfront resort-member itc hotel group,"adi dharma hotel, bali",adya hotel langkawi,al manzel hotel apartments,alchymist grand hotel & spa,alila diwa,alila fort bishangarh,aliya resort and spa,aloha on the ganges by leisure hotels,alpana hotel,alpine ridge,alps resorts,alsisar haveli - a heritage hotel,"ama plantation trails, coorg",amantra shilpi resort spa & club,amanvana spa resort- coorg,"amara grand baga, goa",amara vacanza grand inn,amari watergate hotel bangkok,amaya lake,ambassador - ihcl seleqtions,ambassador ajanta,ambassador hotel bangkok,ambavgarh retreat,amber dale luxury hotel and spa,ambre hotel (adult only property),ambrosia sarovar portico,anand portico,ananta spa & resort,aonang paradise resort krabi,apple valley resort,arcadia heritage resort,arcadia regency,arika villa,arrowtown house boutique accommodation,aseania resort & spa langkawi,ashhok hassan,ashlee plaza patong hotel & spa,atdc houseboat,aura,avangio hotel kota kinabalu managed by accorhotels,avani pattaya,azzure by spree hotels,...,Seeta Amman Temple,Seetha Eliya,Self Drive SUV for 2 days from Delhi,Self Drive SUV for 3 days from Chandigarh,Self Drive SUV for 3 days from Delhi,Self Drive SUV for 3 days from Mumbai,Self Drive SUV for 3 days from Pune,Self Drive SUV for 4 days from Ahmedabad,Self Drive SUV for 4 days from Bangalore,Self Drive SUV for 4 days from Delhi,Self Drive SUV for 4 days from Mumbai,Self Drive SUV for 4 days from Pune,Self Drive SUV for 5 days from Chennai,Self Drive Sedan for 2 days from Delhi,Self Drive Sedan for 3 days from Delhi,Self Drive Sedan for 3 days from Mumbai,Self Drive Sedan for 4 days from Bangalore,Self Drive Sedan for 4 days from Delhi,Sentosa Experiential Tour with Shared Transfers,Seven Sister Falls,Seven Sister Waterfall,Shalimar Garden,Sham Valley,Shani Shingnapur,Shankaracharya Temple,Shanti Stupa,Shared Transfer - Bus Station to Kuala Lumpur City Center Hotel,Shared Transfer - Cruise Terminal to Hotel in Singapore,Shared Transfer - Hotel in Singapore to Cruise Terminal,Shared Trans

In [182]:
print('================VALIDATION DATA==================')
display(final_validation_data)

================VALIDATION DATA==================


,Travel Date,Flight Stops,Meals,0,1,2,3,4,1,2,3,4,5,6,7,8,30-01-2021,Abu Dhabi,Agra,Ahmedabad,Ahmednagar,Ajmer,Alibagh,Allepey,Almora,Alwar,Amritsar,Anuradhapura,Auckland,Aurangabad,Bali,Bandipur,Bangalore,Bangkok,Batam,Bentota,Bharatpur,Bikaner,Bintan Island,Bundi,Cairns,Chail,Chandigarh,Chennai,Cherrapunjee,Chikmangalur,Chintpurni,Chittorgarh,Christchurch,Cochin,Colombo,Coorg,Copenhagen,Corbett,Dalhousie,Daman,Dambulla,Darjeeling,Dharamshala,Dubai,Dwarka,Franz Josef,Galle,Gangtok,Ganpatipule,Genting,Goa,Gokarna,Gold Coast,Gulmarg,Guwahati,Gwalior,Hampi,Haridwar,Hassan,Havelock,Hong Kong,Jaipur,Jaisalmer,Jakarta,Jammu,Jamnagar,Jodhpur,Johor Bahru,Joshimath,Kabini,Kalimpong,Kanchipuram,Kandy,Kangra,Kanyakumari,Kasauli,Kasol,Katra,Kausani,Kaziranga,Khajuraho,Kishangarh,Kodaikanal,Kota Kinabalu,Kovalam,Kovalam and Poovar,Krabi town,Kuala Lumpur,Kullu,Kumarakom,Kumbakonam,Kumbalgarh,Kuta,Lachen,Lachung,Langkawi,Leh,Lonavala And Khandala,Lucknow,Madurai,Mahabalipuram,Mahe Island,Manali,Mandawa,Mathura,Mauritius,Melbourne,Moscow,Mount Abu,Mumbai,Munnar,Mussoorie,Mysore,Nainital,Namchi,Nameri,Nanded,Nasik,Nawalgarh,Negombo,New Delhi,Niel Island,Nubra Valley,Nuwara Eliya,Ooty,Orchha,Pahalgam,Palampur,Pali,Paris,Paro,Parwanoo,Patnitop,Pattaya,Pelling,Penang,Phuentsholing,Phuket,Pondicherry,Poovar,Port Blair,Prague,Praslin Island,Punakha,Pushkar,Queenstown,Rameshwaram,Ranakpur,Ranikhet,Ranthambhore,Ras Al Khaimah,Rishikesh,Rotorua,Saint Petersburg,Saputara,Seminyak,Sharjah,Shillong,Shimla,Shirdi,Sigiriya,Siliguri,Singapore,Somnath,Sonmarg,Srinagar,Sydney,Tanjore,Thekkady,Thimpu,Tiruchirapally,Tirupati,Trincomalee,Trivandrum,Udaipur,Vadodara,Varanasi,Varkala,Wayanad,Yala,9hotel montparnasse,a one star hotel pattaya,aahana - the corbett wilderness,aamod at dalhousie,abhishek beach resort and spa,acorn hideaway resort & spa,acron waterfront resort-member itc hotel group,adya hotel langkawi,al manzel hotel apartments,alchymist grand hotel & spa,alila diwa,alila fort bishangarh,aliya resort and spa,aloha on the ganges by leisure hotels,alpine ridge,alps resorts,"ama plantation trails, coorg",amara grand baga,"amara grand baga, goa",amara vacanza grand inn,amari watergate hotel bangkok,amaya lake,ambassador - ihcl seleqtions,ambassador ajanta,amber dale luxury hotel and spa,ambre hotel (adult only property),ambrosia sarovar portico,anand portico,ananta spa & resort,andaz delhi - a concept by hyatt,apple valley resort,arcadia heritage resort,arcadia regency,arika villa,arrowtown house boutique accommodation,ashhok hassan,atapattu walauwa,atdc houseboat,azzure by spree hotels,backwater retrezt houseboat,baikunth resorts,bandipur safari lodge by jungle lodges,bella vista waterfront,berjaya langkawi resort,best bella pattaya,best western elyon,best western kuta beach,bhanwar singh palace,bhati desert camp bikaner,blue bells valley resort,blue ocean resort & spa,bogmallo beach resort,bonjour bonheur ocean spray,bota royal,...,Scuba Diving Experience in Havelock (With Photos and Video),Scuba Diving and Excursion to Tarkarli Beach (With Transfers) For Guests Staying in North Goa only - MMT,Scuba Diving and Excursion to Tarkarli Beach (With Transfers) For Guests Staying in South Goa only - MMT,Sea Life Ocean World and Madame Tussaud,Seeta Amman Temple,Seetha Eliya,Self Drive SUV for 2 days from Delhi,Self Drive SUV for 3 days from Chandigarh,Self Drive SUV for 3 days from Delhi,Self Drive SUV for 3 days from Mumbai,Self Drive SUV for 3 days from Pune,Self Drive SUV for 4 days from Ahmedabad,Self Drive SUV for 4 days from Bangalore,Self Drive SUV for 4 days from Delhi,Self Drive SUV for 4 days from Mumbai,Self Drive SUV for 4 days from Pune,Self Drive SUV for 5 days from Chennai,Self Drive Sedan for 2 days from Delhi,Self Drive Sedan for 3 days from Delhi,Self Drive Sedan for 3 days from Mumbai,Self Drive Sedan for 4 days from Bangalore,Self Drive Sedan for 4 days from Delhi,Sentosa Experiential Tour with Shared Transfers,Seven Sister Falls,Seven Sis

In [70]:
### Saving processed data to csv

final_train_data.to_csv('dataset\Train_prepared.csv') 
final_validation_data.to_csv('dataset\Test_prepared.csv')